In [1]:
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()
!pip install importlib-metadata==4.12.0 # To overcome an issue with importlib-metadata https://stackoverflow.com/questions/73929564/entrypoints-object-has-no-attribute-get-digital-ocean
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install huggingface_sb3
!pip install pyglet==1.5.1
!pip install ale-py==0.7.4 # To overcome an issue with gym (https://github.com/DLR-RM/stable-baselines3/issues/875)

!pip install pickle5
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 1s (214 kB/s)
Reading package li

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
env_id = "MountainCar-v0"
env = make_vec_env(env_id, 16)
eval_env = make_vec_env(env_id,5)

eval_freq = int(1e5)
eval_freq = max(eval_freq//16, 1)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path = "./logs/",
    eval_freq=eval_freq,
    n_eval_episodes=10
)

In [17]:
model = PPO(
    policy="MlpPolicy",
    env=env,
    gamma=0.99,
    batch_size=512,
    verbose = 1,
    train_freq=16,
    target_update_interval=600,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_starts=1000,
    learning_rate=0.004,
    gradient_steps=8,
    buffer_size=int(1e4),
)

Using cpu device


In [18]:
model.learn(1200000, eval_callback)

model=DQN.load("logs/best_model.zip")

Streaming output truncated to the last 5000 lines.
|    n_updates        | 25008    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 194      |
|    ep_rew_mean      | -194     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 4028     |
|    fps              | 3876     |
|    time_elapsed     | 206      |
|    total_timesteps  | 801312   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 2.68e-06 |
|    n_updates        | 25016    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 194      |
|    ep_rew_mean      | -194     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 4032     |
|    fps              | 3876     |
|    time_elapsed     | 206      |
|    total_timesteps  | 801568   |
| tr

KeyboardInterrupt: ignored

In [13]:
eval_env = gym.make(env_id)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-200.00 +/- 0.0


In [7]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender
!pip install imageio==2.4.1

from colabgymrender.recorder import Recorder

dir = "./video"
env = Recorder(eval_env, dir)
obs = env.reset()
done=False
while not done:
  action, state = model.predict(obs)
  obs,reward,done,info = env.step(action)
env.play()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
